In [1]:
import sys
#BASE_DIR = "/home/dzigen/Desktop/ITMO/smiles2024/RAG-project-SMILES-2024-"
BASE_DIR = "/trinity/home/team06/workspace/mikhail_workspace/rag_project"
sys.path.insert(0, BASE_DIR)

import pandas as pd 
import ast
import numpy as np
import json
from tqdm import tqdm
from IPython.display import clear_output
import os
from typing import Dict, List

from src.Retriever import ThresholdRetriever
from src.Scorer import SimilarityScorerConfig
from src.Reader import LLM_Model
from src.utils import ReaderMetrics, RetrieverMetrics, save_rag_trial_log, prepare_thresholdretriever_configs, prepare_reader_configs, load_benchmarks_df
from src.utils import evaluate_retriever, evaluate_reader

/trinity/home/team06/workspace/mikhail_workspace/mikhail_venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# !!! TO CHANGE !!!
TRIAL = 1
BENCHMARKS_MAXSIZE = 500
BENCHMARKS_INFO = {'mtssquad': {'db': 'v3', 'table': 'v2'}}

# Retriever part
RETRIEVER_PARAMS = {
    "model_path": f"{BASE_DIR}/models/intfloat/multilingual-e5-small",
    "densedb_kwargs": {'metadata': {"hnsw:space": "ip"}},
    "model_kwargs": {'device':'cuda'},
    "encode_kwargs": {'normalize_embeddings': True, 'prompt': 'query: '},
    "params": {'fetch_k': 50, 'threshold': 0.23, 'max_k': 3}
}

# Reader part
READER_PARAMS = {
    'prompts': {
        "assistant": 'Отвечай на вопросы, используя информацию из текстов в списке ниже. Каждому тексту в начале в квадратных скобках поставлена в соответствие вещественная оценка его синтаксической сложности: в диапазоне от 0.0 (низкая сложность) до 1.0 (высокая сложность). Учитывай эти данные. Тексты с более низкой оценкой содержат меньше дополнительной информации и из них легче извлечь релевантную информацию, если такая есть. Если оценка кажется тебе слишком высокой, а другие тексты нерелевантны запросу,  то сгенерируй следующий ответ: "У меня нет ответа на ваш вопрос.".',
        "system": "Ты вопросно-ответная система. Все ответы генерируй на русском языке. По вопросам отвечай кратко, чётко и конкретно. Не генерируй излишнюю информацию.",
    },
    'gen': {'max_new_tokens': 512, 'eos_token_id': 79097},
    'data_operate': {'batch_size': 1}
    }

ADDITIONAL_PARAMS = {
    'topk_score_list': 3
}

BERTSCORE_MODEL_PATH = "ru_electra_medium"
# !!! TO CHANGE !!!

SAVE_LOGDIR = f'./logs/trial{TRIAL}'
SAVE_HYPERPARAMS = f'{SAVE_LOGDIR}/hyperparams.json'
SAVE_READERCACHE = f'{SAVE_LOGDIR}/reader_cache.json'
SAVE_RETRIEVERCACHE = f'{SAVE_LOGDIR}/retriever_cache.json'

##### Configure Retriever-part

In [3]:
retrievers_config, benchmarks_path = prepare_thresholdretriever_configs(BASE_DIR, BENCHMARKS_INFO, RETRIEVER_PARAMS)
retriever_metrics = RetrieverMetrics()

In [4]:
benchmarks_df = load_benchmarks_df(benchmarks_path, BENCHMARKS_MAXSIZE)

##### Configure Reader-part

In [5]:
reader_config = prepare_reader_configs(READER_PARAMS)
READER = LLM_Model(reader_config)

Loading checkpoint shards: 100%|██████████| 4/4 [03:38<00:00, 54.70s/it]


In [6]:
sim_score_config = SimilarityScorerConfig()
reader_metrics = ReaderMetrics(BASE_DIR, BERTSCORE_MODEL_PATH, sim_score_config, READER)

Loading Meteor...
Loading ExactMatch


In [7]:
RETRIEVERS = {name: ThresholdRetriever(config) for name, config in retrievers_config.items()}

No sentence-transformers model found with name /trinity/home/team06/workspace/mikhail_workspace/rag_project/models/intfloat/multilingual-e5-small. Creating a new one with mean pooling.


##### Evaluating pipeline

In [8]:
retriever_scores, retriever_cache_ids, predicted_chunks, cache_relevant_flags = evaluate_retriever(
    benchmarks_df, RETRIEVERS, retriever_metrics, show_step=5, topk_score_list=ADDITIONAL_PARAMS['topk_score_list'])

mtssquad


  0%|          | 0/500 [00:00<?, ?it/s]/trinity/home/team06/workspace/mikhail_workspace/rag_project/src/utils/evaluation_metrics.py:42: RuntimeWarning: invalid value encountered in scalar divide
  return (2 * self.precision(pred_cands, gold_cands, k) * self.recall(pred_cands, gold_cands, k)) / (self.precision(pred_cands, gold_cands, k) + self.recall(pred_cands, gold_cands, k))
/trinity/home/team06/workspace/mikhail_workspace/mikhail_venv/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/trinity/home/team06/workspace/mikhail_workspace/mikhail_venv/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
100%|██████████| 500/500 [00:11<00:00, 42.26it/s, MRR=0.778, mAP=0.778, Recall=0.839, Precision=0.28, F1=0.419, NoRelContextScore=nan] 


In [14]:
contexts = {name: [reader_config.prompts.assistant + "\n\n" + "\n\n".join([f"{i+1}. [{round(doc[2]['anna_score'], 3)}] {doc[1]}" for i, doc in enumerate(docs)]) 
                                                                      for docs in chunks] for name, chunks in predicted_chunks.items()}

In [18]:
reader_scores, reader_cache = evaluate_reader(benchmarks_df, READER, reader_metrics, contexts, show_step=1, cache_relevant_flags=cache_relevant_flags)

0it [00:00, ?it/s]

answer raw len:  0 1


1it [00:09,  9.37s/it, BLEU2=nan, BLEU1=nan, ExactMatch=nan, METEOR=nan, BertScore=nan, StubScore=0]

sim raw len:  1
False
Нет, Мадонна не просила изменить аранжировку песни.
Да, по просьбе Мадонны заменили аранжировщика Каминса на более опытного штатного аранжировщика Warner Bros. Records Регги Лукаса.
[0]
answer raw len:  1 1


2it [00:11,  5.32s/it, BLEU2=nan, BLEU1=nan, ExactMatch=nan, METEOR=nan, BertScore=nan, StubScore=0]

...

answer raw len:  497 1
True
Александр Грин был сослан в Пинегу, Архангельскую губернию, после третьего ареста.
После третьего ареста писатель Александр Грин был сослан в Пинегу Архангельской губернии.
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0]


499it [28:19,  4.29s/it, BLEU2=0.349, BLEU1=0.422, ExactMatch=0.0358, METEOR=0.608, BertScore=nan, StubScore=0.2]

answer raw len:  498 1
True
Отвечаю на вопрос: [0.04] 27 июня Викинг оказался затерт сплошными ледяными полями, начался дрейф. Шхуна была снесена на запад, иногда оказывалась на расстоянии 12-13 миль от гренландского побережья. Исследуя айсберги в начале июля, Нансен обнаружил, что они могут переносить значительные количества минерального материала, что дает представление о геологическом строении неизведанных областей Гренландии.

Ответ: Нансен.
О том, что айсберги могут переносить значительные количества минерального материала, дающего представление о геологическом строении неисследованных областей Гренландии, обнаружил Фритьоф Нансен.
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0]


500it [28:21,  3.40s/it, BLEU2=0.349, BLEU1=0.422, ExactMatch=0.0357, METEOR=0.608, BertScore=nan, StubScore=0.2]

answer raw len:  499 1
True
27 июня Викинг оказался заперт в сплошных льдах и начался дрейф.
27 июня Викинг Нансена оказался затёрт сплошными ледовыми полями, начался незапланированный дрейф.
[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0]


In [19]:
save_rag_trial_log(SAVE_LOGDIR, reader_scores, retriever_scores, 
                   SAVE_HYPERPARAMS, SAVE_READERCACHE, SAVE_RETRIEVERCACHE,
                   reader_cache, retriever_cache_ids, BENCHMARKS_INFO, BENCHMARKS_MAXSIZE,
                   READER_PARAMS, RETRIEVER_PARAMS, ADDITIONAL_PARAMS)

In [20]:
import gc
import torch

gc.collect()
torch.cuda.empty_cache()